In [1]:
"""
===============================================================================
NOTEBOOK 02: DATA CLEANING & FEATURE ENGINEERING FOR EMERGENCY DISPATCH
===============================================================================

Purpose: Create dispatch-time available features for accident severity prediction

Critical Principle: ONLY use information available WHEN THE EMERGENCY CALL COMES IN
- Location (GPS coordinates) 
- Timestamp (when call received) 
- Historical patterns at location/time 

Author: Mary Wangoi Mwangi (122174)
Supervisor: Prof. Vincent Omwenga
Date: January 2026
===============================================================================
"""

# ============================================================================
# IMPORTS
# ============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("="*70)
print("NOTEBOOK 02: DATA CLEANING & FEATURE ENGINEERING")
print("="*70)
print("\n Libraries imported successfully")
print("  - pandas, numpy: Data manipulation")
print("  - matplotlib, seaborn: Visualization")
print("  - sklearn: Data splitting")
print("  - imblearn: SMOTE for class imbalance")



# ============================================================================
# LOAD LABELED DATASET
# ============================================================================
print("\n" + "="*70)
print("LOADING LABELED DATASET FROM NOTEBOOK 01")
print("="*70)

# Load the dataset created in Notebook 01
data_path = '../data/processed/labeled_crashes.csv'
df = pd.read_csv(data_path)

# Convert datetime columns
df['crash_datetime'] = pd.to_datetime(df['crash_datetime'])
df['crash_date'] = pd.to_datetime(df['crash_date'])

# Display basic info
print(f"\nDataset loaded successfully!")
print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"Date range: {df['crash_date'].min().date()} to {df['crash_date'].max().date()}")

# Display column categories
print("\n" + "="*70)
print("FEATURE CATEGORIES FROM NOTEBOOK 01")
print("="*70)

print("\n1. ORIGINAL WORLD BANK FEATURES (10 columns):")
original_cols = ['crash_id', 'crash_datetime', 'crash_date', 'latitude', 'longitude', 
                 'n_crash_reports', 'contains_fatality_words', 'contains_pedestrian_words',
                 'contains_matatu_words', 'contains_motorcycle_words']
for col in original_cols:
    print(f"   - {col}")

print("\n2. SEVERITY LABELS (2 columns):")
severity_cols = ['severity_4class', 'severity_binary']
for col in severity_cols:
    print(f"   - {col}")

print("\n3. TEMPORAL FEATURES (6 columns):")
temporal_cols = ['hour', 'day_of_week', 'day_name', 'month', 'year', 'is_weekend']
for col in temporal_cols:
    print(f"   - {col}")

print("\n4. SPATIAL FEATURES (6 columns):")
spatial_cols = ['lat_bin', 'lon_bin', 'grid_cell', 'crashes_at_location', 
                'high_rate_at_location', 'location_risk_category']
for col in spatial_cols:
    print(f"   - {col}")
    

# Display target variable distribution
print("\n" + "="*70)
print("TARGET VARIABLE: BINARY SEVERITY")
print("="*70)
print(df['severity_binary'].value_counts())
print(f"\nClass imbalance ratio (LOW:HIGH): {(df['severity_binary']=='LOW').sum() / (df['severity_binary']=='HIGH').sum():.2f}:1")

print("\n Data loaded successfully")
print(" Ready for feature engineering")

NOTEBOOK 02: DATA CLEANING & FEATURE ENGINEERING

 Libraries imported successfully
  - pandas, numpy: Data manipulation
  - matplotlib, seaborn: Visualization
  - sklearn: Data splitting
  - imblearn: SMOTE for class imbalance

LOADING LABELED DATASET FROM NOTEBOOK 01

Dataset loaded successfully!
Shape: 31,064 rows × 24 columns
Memory usage: 12.87 MB
Date range: 2012-08-08 to 2023-07-12

FEATURE CATEGORIES FROM NOTEBOOK 01

1. ORIGINAL WORLD BANK FEATURES (10 columns):
   - crash_id
   - crash_datetime
   - crash_date
   - latitude
   - longitude
   - n_crash_reports
   - contains_fatality_words
   - contains_pedestrian_words
   - contains_matatu_words
   - contains_motorcycle_words

2. SEVERITY LABELS (2 columns):
   - severity_4class
   - severity_binary

3. TEMPORAL FEATURES (6 columns):
   - hour
   - day_of_week
   - day_name
   - month
   - year
   - is_weekend

4. SPATIAL FEATURES (6 columns):
   - lat_bin
   - lon_bin
   - grid_cell
   - crashes_at_location
   - high_rate_at_l

In [2]:
"""
SECTION 2: Missing Value Analysis & Treatment

Objective: Identify, understand, and handle missing values appropriately.
Focus: The 1,211 missing values in location_risk_category from Notebook 01.
"""

print("="*70)
print("MISSING VALUE ANALYSIS")
print("="*70)

# Check for missing values across all columns
missing_summary = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2)
})

missing_summary = missing_summary[missing_summary['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

if len(missing_summary) > 0:
    print("\n Columns with missing values:")
    print(missing_summary.to_string(index=False))
else:
    print("\n No missing values found!")

    

# ============================================================================
# INVESTIGATE location_risk_category MISSING VALUES
# ============================================================================
print("\n" + "="*70)
print("INVESTIGATING location_risk_category MISSING VALUES")
print("="*70)

# Find which crashes have missing risk categories
missing_risk = df[df['location_risk_category'].isnull()]

print(f"\nCrashes with missing location_risk_category: {len(missing_risk):,}")
print(f"Percentage of total dataset: {len(missing_risk)/len(df)*100:.2f}%")

# Analyze the high_rate_at_location for missing cases
print("\nWhy are these missing?")
print(f"Min high_rate_at_location (missing cases): {missing_risk['high_rate_at_location'].min():.2f}%")
print(f"Max high_rate_at_location (missing cases): {missing_risk['high_rate_at_location'].max():.2f}%")
print(f"Mean high_rate_at_location (missing cases): {missing_risk['high_rate_at_location'].mean():.2f}%")

print("\nOur risk category bins (from Notebook 01):")
print("  - LOW_RISK: 0-10%")
print("  - MEDIUM_RISK: 10-15%")
print("  - HIGH_RISK: 15-20%")
print("  - VERY_HIGH_RISK: 20-100%")

# The issue: values below 10% or exactly at boundaries
print("\n ISSUE IDENTIFIED:")
print("   Missing values occur when high_rate_at_location is EXACTLY 0%")
print("   or falls outside bin boundaries (edge case handling)")



# ============================================================================
# FIX MISSING VALUES
# ============================================================================
print("\n" + "="*70)
print("FIXING MISSING VALUES IN location_risk_category")
print("="*70)

# Strategy: Create a more inclusive binning that covers ALL cases
df['location_risk_category'] = pd.cut(
    df['high_rate_at_location'], 
    bins=[-0.01, 10, 15, 20, 100],  # Include -0.01 to catch 0% cases
    labels=['LOW_RISK', 'MEDIUM_RISK', 'HIGH_RISK', 'VERY_HIGH_RISK'],
    include_lowest=True  # Include the lowest boundary
)

# Verify fix
remaining_missing = df['location_risk_category'].isnull().sum()
print(f"\nMissing values after fix: {remaining_missing}")

if remaining_missing == 0:
    print(" All missing values successfully fixed!")
else:
    print(f" Still {remaining_missing} missing values - will fill with 'LOW_RISK'")
    df['location_risk_category'].fillna('LOW_RISK', inplace=True)

# Display updated distribution
print("\n" + "="*70)
print("UPDATED location_risk_category DISTRIBUTION")
print("="*70)
print(df['location_risk_category'].value_counts().sort_index())



# ============================================================================
# FINAL MISSING VALUE CHECK
# ============================================================================
print("\n" + "="*70)
print("FINAL MISSING VALUE CHECK")
print("="*70)

total_missing = df.isnull().sum().sum()
if total_missing == 0:
    print(" No missing values in dataset!")
    print(" Dataset is complete and ready for feature engineering")
else:
    print(f" {total_missing} missing values remaining")
    print("\n Columns with missing values:")
    print(df.isnull().sum()[df.isnull().sum() > 0])

print(f"\n Missing value treatment complete")
print(f" Dataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

MISSING VALUE ANALYSIS

 Columns with missing values:
                Column  Missing_Count  Missing_Percentage
location_risk_category           1211                 3.9

INVESTIGATING location_risk_category MISSING VALUES

Crashes with missing location_risk_category: 1,211
Percentage of total dataset: 3.90%

Why are these missing?
Min high_rate_at_location (missing cases): 0.00%
Max high_rate_at_location (missing cases): 0.00%
Mean high_rate_at_location (missing cases): 0.00%

Our risk category bins (from Notebook 01):
  - LOW_RISK: 0-10%
  - MEDIUM_RISK: 10-15%
  - HIGH_RISK: 15-20%
  - VERY_HIGH_RISK: 20-100%

 ISSUE IDENTIFIED:
   Missing values occur when high_rate_at_location is EXACTLY 0%
   or falls outside bin boundaries (edge case handling)

FIXING MISSING VALUES IN location_risk_category

Missing values after fix: 0
 All missing values successfully fixed!

UPDATED location_risk_category DISTRIBUTION
location_risk_category
LOW_RISK          10308
MEDIUM_RISK       13029
HIGH_

In [3]:
"""
SECTION 3: Remove Data Leakage Features (CRITICAL)

Data leakage occurs when features used for prediction contain information
that would NOT be available at the time predictions need to be made.

In emergency dispatch, we must ONLY use information available WHEN THE CALL COMES IN.

AVAILABLE at dispatch time:
  GPS location (caller provides)
  Call timestamp (automatic)
  Historical patterns at location/time

NOT AVAILABLE at dispatch time (DATA LEAKAGE):
  Keyword flags (contains_fatality_words, etc.) - from crash reports AFTER accident
  Number of reports (n_crash_reports) - only known after multiple people report
  Crash description text - not available until scene assessment

These features create artificially high model performance that won't work in real dispatch!
"""

print("="*70)
print("IDENTIFYING DATA LEAKAGE FEATURES")
print("="*70)

# ============================================================================
# IDENTIFY LEAKAGE FEATURES
# ============================================================================

# Features that contain post-accident information
leakage_features = [
    'n_crash_reports',           # Only known after reports accumulate
    'contains_fatality_words',   # From crash scene reports
    'contains_pedestrian_words', # From crash scene reports
    'contains_matatu_words',     # From crash scene reports
    'contains_motorcycle_words'  # From crash scene reports
]

print("\n DATA LEAKAGE FEATURES (Must be removed):")
print("-" * 70)
for i, feature in enumerate(leakage_features, 1):
    print(f"{i}. {feature}")
    print(f"   Why leakage? This information comes from crash reports AFTER the accident")
    print(f"   Dispatcher doesn't have this when emergency call comes in\n")

# Show correlation with target (they'll be suspiciously high!)
print("=" * 70)
print("CORRELATION WITH TARGET (Showing why these are leakage)")
print("=" * 70)

# Convert binary target to numeric for correlation
df['severity_numeric'] = (df['severity_binary'] == 'HIGH').astype(int)

print("\nCorrelation with HIGH severity:")
print("-" * 70)
for feature in leakage_features:
    corr = df[feature].corr(df['severity_numeric'])
    print(f"{feature:30s}: {corr:6.3f} {'← SUSPICIOUSLY HIGH!' if abs(corr) > 0.3 else ''}")

# Show how keyword flags relate to our severity labels
print("\n" + "=" * 70)
print("PROOF OF DATA LEAKAGE: Keyword Flags vs Severity Labels")
print("=" * 70)

print("\n1. contains_fatality_words vs severity_binary:")
print(pd.crosstab(df['contains_fatality_words'], df['severity_binary'], 
                  normalize='index') * 100)

print("\n2. contains_pedestrian_words vs severity_binary:")
print(pd.crosstab(df['contains_pedestrian_words'], df['severity_binary'], 
                  normalize='index') * 100)

print("\n3. contains_motorcycle_words vs severity_binary:")
print(pd.crosstab(df['contains_motorcycle_words'], df['severity_binary'], 
                  normalize='index') * 100)

print("\n NOTICE: Keyword flags are DIRECT INDICATORS of severity!")
print("   Using these features would be circular reasoning, not prediction.")



# ============================================================================
# REMOVE LEAKAGE FEATURES
# ============================================================================

print("\n" + "=" * 70)
print("REMOVING DATA LEAKAGE FEATURES")
print("=" * 70)

print(f"\nBefore removal: {len(df.columns)} columns")

# Drop leakage features
df_clean = df.drop(columns=leakage_features)

# Also drop the temporary severity_numeric column
df_clean = df_clean.drop(columns=['severity_numeric'])

print(f"After removal:  {len(df_clean.columns)} columns")
print(f"Removed:        {len(leakage_features)} leakage features")



# ============================================================================
# VERIFY REMAINING FEATURES
# ============================================================================

print("\n" + "=" * 70)
print("REMAINING FEATURES (All dispatch-time available)")
print("=" * 70)

# Categorize remaining features
dispatch_available = {
    'Identifiers (3)': ['crash_id', 'crash_datetime', 'crash_date'],
    'Location (2)': ['latitude', 'longitude'],
    'Temporal (6)': ['hour', 'day_of_week', 'day_name', 'month', 'year', 'is_weekend'],
    'Spatial (6)': ['lat_bin', 'lon_bin', 'grid_cell', 'crashes_at_location', 
                    'high_rate_at_location', 'location_risk_category'],
    'Target (2)': ['severity_4class', 'severity_binary']
}

for category, features in dispatch_available.items():
    print(f"\n{category}:")
    for feature in features:
        if feature in df_clean.columns:
            print(f"  ✓ {feature}")

print("\n" + "=" * 70)
print("DATA LEAKAGE REMOVAL COMPLETE")
print("=" * 70)

print(f"\n Removed {len(leakage_features)} features with post-accident information")
print(f" Retained {len(df_clean.columns)} dispatch-time available features")
print(f" Dataset shape: {df_clean.shape[0]:,} rows × {df_clean.shape[1]} columns")
print("\n Model will now be trained ONLY on information available at dispatch time")
print("  This ensures realistic performance estimates for operational deployment")

# Update df to clean version
df = df_clean.copy()

IDENTIFYING DATA LEAKAGE FEATURES

 DATA LEAKAGE FEATURES (Must be removed):
----------------------------------------------------------------------
1. n_crash_reports
   Why leakage? This information comes from crash reports AFTER the accident
   Dispatcher doesn't have this when emergency call comes in

2. contains_fatality_words
   Why leakage? This information comes from crash reports AFTER the accident
   Dispatcher doesn't have this when emergency call comes in

3. contains_pedestrian_words
   Why leakage? This information comes from crash reports AFTER the accident
   Dispatcher doesn't have this when emergency call comes in

4. contains_matatu_words
   Why leakage? This information comes from crash reports AFTER the accident
   Dispatcher doesn't have this when emergency call comes in

5. contains_motorcycle_words
   Why leakage? This information comes from crash reports AFTER the accident
   Dispatcher doesn't have this when emergency call comes in

CORRELATION WITH TARGET (Sho

In [4]:
"""
SECTION 4: Engineer Dispatch-Time Features

Create features that predict severity using ONLY information available
when the emergency call comes in.

Feature Engineering Strategy:
1. Temporal Risk Features: Historical severity rates by hour/day/month
2. Spatial Risk Features: Already have (crashes_at_location, high_rate_at_location)
3. Interaction Features: Location × Time patterns
4. Categorical Encodings: Risk categories and time periods

These features capture patterns like:
- "Accidents at 11 PM are usually more severe"
- "This location has high historical severity"
- "Weekend nights at this location are dangerous"
"""

print("="*70)
print("DISPATCH-TIME FEATURE ENGINEERING")
print("="*70)

# ============================================================================
# TEMPORAL RISK FEATURES (Historical Severity Rates)
# ============================================================================
print("\n" + "="*70)
print("CREATING TEMPORAL RISK FEATURES")
print("="*70)

# Calculate historical HIGH severity rate by hour
hour_severity_rate = df.groupby('hour')['severity_binary'].apply(
    lambda x: (x == 'HIGH').sum() / len(x) * 100
).to_dict()

df['hour_severity_rate'] = df['hour'].map(hour_severity_rate)

print("\n1. hour_severity_rate: Historical HIGH severity % for this hour")
print(f"   Range: {df['hour_severity_rate'].min():.2f}% to {df['hour_severity_rate'].max():.2f}%")
print(f"   Mean: {df['hour_severity_rate'].mean():.2f}%")

# Calculate historical HIGH severity rate by day of week
day_severity_rate = df.groupby('day_of_week')['severity_binary'].apply(
    lambda x: (x == 'HIGH').sum() / len(x) * 100
).to_dict()

df['day_severity_rate'] = df['day_of_week'].map(day_severity_rate)

print("\n2. day_severity_rate: Historical HIGH severity % for this day of week")
print(f"   Range: {df['day_severity_rate'].min():.2f}% to {df['day_severity_rate'].max():.2f}%")
print(f"   Mean: {df['day_severity_rate'].mean():.2f}%")

# Calculate historical HIGH severity rate by month
month_severity_rate = df.groupby('month')['severity_binary'].apply(
    lambda x: (x == 'HIGH').sum() / len(x) * 100
).to_dict()

df['month_severity_rate'] = df['month'].map(month_severity_rate)

print("\n3. month_severity_rate: Historical HIGH severity % for this month")
print(f"   Range: {df['month_severity_rate'].min():.2f}% to {df['month_severity_rate'].max():.2f}%")
print(f"   Mean: {df['month_severity_rate'].mean():.2f}%")



# ============================================================================
# TIME PERIOD INDICATORS
# ============================================================================
print("\n" + "="*70)
print("CREATING TIME PERIOD INDICATORS")
print("="*70)

# Night indicator (highest severity hours: 10 PM - 4 AM)
df['is_night'] = ((df['hour'] >= 22) | (df['hour'] <= 4)).astype(int)
night_high_rate = (df[df['is_night'] == 1]['severity_binary'] == 'HIGH').mean() * 100
day_high_rate = (df[df['is_night'] == 0]['severity_binary'] == 'HIGH').mean() * 100

print(f"\n4. is_night (10 PM - 4 AM):")
print(f"   Night crashes: {df['is_night'].sum():,} ({df['is_night'].mean()*100:.1f}%)")
print(f"   Night HIGH rate: {night_high_rate:.2f}%")
print(f"   Day HIGH rate: {day_high_rate:.2f}%")
print(f"   Night is {night_high_rate/day_high_rate:.2f}x more dangerous")

# Rush hour indicator (morning: 6-9 AM, evening: 5-7 PM)
df['is_rush_hour'] = (((df['hour'] >= 6) & (df['hour'] <= 9)) | 
                      ((df['hour'] >= 17) & (df['hour'] <= 19))).astype(int)
rush_high_rate = (df[df['is_rush_hour'] == 1]['severity_binary'] == 'HIGH').mean() * 100
non_rush_high_rate = (df[df['is_rush_hour'] == 0]['severity_binary'] == 'HIGH').mean() * 100

print(f"\n5. is_rush_hour (6-9 AM, 5-7 PM):")
print(f"   Rush hour crashes: {df['is_rush_hour'].sum():,} ({df['is_rush_hour'].mean()*100:.1f}%)")
print(f"   Rush hour HIGH rate: {rush_high_rate:.2f}%")
print(f"   Non-rush HIGH rate: {non_rush_high_rate:.2f}%")



# ============================================================================
# LOCATION × TIME INTERACTION FEATURES
# ============================================================================
print("\n" + "="*70)
print("CREATING LOCATION × TIME INTERACTION FEATURES")
print("="*70)

# High-risk location at dangerous time
df['high_risk_location'] = (df['high_rate_at_location'] > 15).astype(int)
df['dangerous_time'] = ((df['is_night'] == 1) | (df['is_weekend'] == 1)).astype(int)
df['high_risk_location_dangerous_time'] = df['high_risk_location'] * df['dangerous_time']

interaction_high_rate = (df[df['high_risk_location_dangerous_time'] == 1]['severity_binary'] == 'HIGH').mean() * 100
baseline_high_rate = (df[df['high_risk_location_dangerous_time'] == 0]['severity_binary'] == 'HIGH').mean() * 100

print(f"\n6. high_risk_location_dangerous_time:")
print(f"   (High-risk location × Night/Weekend)")
print(f"   Interaction cases: {df['high_risk_location_dangerous_time'].sum():,}")
print(f"   HIGH rate when both: {interaction_high_rate:.2f}%")
print(f"   HIGH rate baseline: {baseline_high_rate:.2f}%")
print(f"   Risk multiplier: {interaction_high_rate/baseline_high_rate:.2f}x")



# ============================================================================
# SUMMARY OF NEW FEATURES
# ============================================================================
print("\n" + "="*70)
print("FEATURE ENGINEERING SUMMARY")
print("="*70)

new_features = [
    'hour_severity_rate',
    'day_severity_rate', 
    'month_severity_rate',
    'is_night',
    'is_rush_hour',
    'high_risk_location',
    'dangerous_time',
    'high_risk_location_dangerous_time'
]

print(f"\nCreated {len(new_features)} new dispatch-time features:")
for i, feature in enumerate(new_features, 1):
    print(f"  {i}. {feature}")

print(f"\nTotal features now: {len(df.columns)}")
print(f"  - Identifiers: 3")
print(f"  - Location: 2 (latitude, longitude)")
print(f"  - Temporal original: 6")
print(f"  - Temporal engineered: 3 (severity rates)")
print(f"  - Temporal indicators: 2 (night, rush_hour)")
print(f"  - Spatial original: 6")
print(f"  - Spatial engineered: 1 (high_risk_location)")
print(f"  - Interaction: 2")
print(f"  - Target: 2")

print("\n Feature engineering complete")
print(" All features are dispatch-time available")
print(" Ready for train/validation/test split")

DISPATCH-TIME FEATURE ENGINEERING

CREATING TEMPORAL RISK FEATURES

1. hour_severity_rate: Historical HIGH severity % for this hour
   Range: 9.57% to 17.50%
   Mean: 12.50%

2. day_severity_rate: Historical HIGH severity % for this day of week
   Range: 11.29% to 15.16%
   Mean: 12.50%

3. month_severity_rate: Historical HIGH severity % for this month
   Range: 10.85% to 13.47%
   Mean: 12.50%

CREATING TIME PERIOD INDICATORS

4. is_night (10 PM - 4 AM):
   Night crashes: 2,115 (6.8%)
   Night HIGH rate: 14.75%
   Day HIGH rate: 12.34%
   Night is 1.20x more dangerous

5. is_rush_hour (6-9 AM, 5-7 PM):
   Rush hour crashes: 15,590 (50.2%)
   Rush hour HIGH rate: 11.34%
   Non-rush HIGH rate: 13.67%

CREATING LOCATION × TIME INTERACTION FEATURES

6. high_risk_location_dangerous_time:
   (High-risk location × Night/Weekend)
   Interaction cases: 2,313
   HIGH rate when both: 20.41%
   HIGH rate baseline: 11.87%
   Risk multiplier: 1.72x

FEATURE ENGINEERING SUMMARY

Created 8 new dispat

In [5]:
"""
SECTION 5A: Weather Feature Engineering

Weather conditions significantly impact accident severity:
- Rain → reduced visibility, slippery roads
- Night + rain → compounded risk
- Temperature extremes → driver fatigue, vehicle issues

Data Source Strategy:
11 years of historical data (2012-2023), I'll create weather-proxy features based on:

1. Season (rainy vs dry season in Nairobi)
2. Time-of-day visibility (daylight vs darkness)
3. Month-specific rain patterns

Note: In operational deployment, these would come from weather API at dispatch time. 
      For historical analysis, I derive from temporal patterns.

Nairobi Climate:
- Long rains: March-May
- Short rains: October-December
- Dry seasons: January-February, June-September
"""

print("="*70)
print("WEATHER FEATURE ENGINEERING")
print("="*70)

# ============================================================================
# SEASON-BASED WEATHER PROXY
# ============================================================================
print("\n" + "="*70)
print("CREATING SEASONAL WEATHER FEATURES")
print("="*70)

def get_season(month):
    """
    Classify month into Nairobi's seasonal patterns
    """
    if month in [3, 4, 5]:
        return 'LONG_RAINS'
    elif month in [10, 11, 12]:
        return 'SHORT_RAINS'
    else:
        return 'DRY_SEASON'

def get_rain_risk(month):
    """
    Rain probability by month (0-1 scale)
    Based on Nairobi historical rainfall patterns
    """
    rain_risk_map = {
        1: 0.15,  # Jan - Dry
        2: 0.20,  # Feb - Dry
        3: 0.70,  # Mar - Long rains peak
        4: 0.85,  # Apr - Long rains peak
        5: 0.65,  # May - Long rains
        6: 0.25,  # Jun - Dry
        7: 0.20,  # Jul - Dry
        8: 0.25,  # Aug - Dry
        9: 0.30,  # Sep - Transition
        10: 0.60, # Oct - Short rains
        11: 0.75, # Nov - Short rains peak
        12: 0.55  # Dec - Short rains
    }
    return rain_risk_map[month]

# Create season feature
df['season'] = df['month'].apply(get_season)

# Create rain risk score
df['rain_risk_score'] = df['month'].apply(get_rain_risk)

print("\n1. season (Nairobi seasonal patterns):")
print(df['season'].value_counts().sort_index())

print("\n2. rain_risk_score (0-1, higher = more likely rain):")
print(f"   Range: {df['rain_risk_score'].min():.2f} to {df['rain_risk_score'].max():.2f}")
print(f"   Mean: {df['rain_risk_score'].mean():.2f}")

# Analyze severity by season
season_severity = df.groupby('season')['severity_binary'].apply(
    lambda x: (x == 'HIGH').sum() / len(x) * 100
)
print("\n   HIGH severity rate by season:")
for season in ['DRY_SEASON', 'LONG_RAINS', 'SHORT_RAINS']:
    if season in season_severity.index:
        print(f"   - {season:15s}: {season_severity[season]:.2f}%")


        
# ============================================================================
# VISIBILITY FEATURES
# ============================================================================
print("\n" + "="*70)
print("CREATING VISIBILITY FEATURES")
print("="*70)

def get_daylight_status(hour, month):
    """
    Estimate daylight vs darkness in Nairobi
    Nairobi (near equator): sunrise ~6:15 AM, sunset ~6:45 PM (varies slightly)
    """
    # Nairobi is near equator, so sunrise/sunset very consistent
    if 7 <= hour <= 18:
        return 'DAYLIGHT'
    else:
        return 'DARKNESS'

df['daylight_status'] = df.apply(lambda row: get_daylight_status(row['hour'], row['month']), axis=1)

print("\n3. daylight_status (visibility conditions):")
print(df['daylight_status'].value_counts())

# Analyze severity by daylight
daylight_severity = df.groupby('daylight_status')['severity_binary'].apply(
    lambda x: (x == 'HIGH').sum() / len(x) * 100
)
print("\n   HIGH severity rate by visibility:")
for status in ['DAYLIGHT', 'DARKNESS']:
    print(f"   - {status:10s}: {daylight_severity[status]:.2f}%")



# ============================================================================
# COMPOUND RISK: RAIN + DARKNESS
# ============================================================================
print("\n" + "="*70)
print("CREATING COMPOUND WEATHER RISK FEATURES")
print("="*70)

# High rain risk (>60% probability)
df['high_rain_risk'] = (df['rain_risk_score'] > 0.6).astype(int)

# Rain + darkness compound risk
df['rain_darkness_risk'] = ((df['high_rain_risk'] == 1) & 
                             (df['daylight_status'] == 'DARKNESS')).astype(int)

compound_risk_high_rate = (df[df['rain_darkness_risk'] == 1]['severity_binary'] == 'HIGH').mean() * 100
baseline_high_rate = (df[df['rain_darkness_risk'] == 0]['severity_binary'] == 'HIGH').mean() * 100

print("\n4. rain_darkness_risk (rainy season + night):")
print(f"   Compound risk cases: {df['rain_darkness_risk'].sum():,}")
print(f"   HIGH rate with compound risk: {compound_risk_high_rate:.2f}%")
print(f"   HIGH rate baseline: {baseline_high_rate:.2f}%")
if baseline_high_rate > 0:
    print(f"   Risk multiplier: {compound_risk_high_rate/baseline_high_rate:.2f}x")



# ============================================================================
# SUMMARY OF WEATHER FEATURES
# ============================================================================
print("\n" + "="*70)
print("WEATHER FEATURES SUMMARY")
print("="*70)

weather_features = [
    'season',
    'rain_risk_score',
    'daylight_status',
    'high_rain_risk',
    'rain_darkness_risk'
]

print(f"\nCreated {len(weather_features)} weather features:")
for i, feature in enumerate(weather_features, 1):
    print(f"  {i}. {feature}")

print(f"\n Weather features engineered")
print(f" Total features now: {len(df.columns)}")
print(f" Ready for road infrastructure features")

WEATHER FEATURE ENGINEERING

CREATING SEASONAL WEATHER FEATURES

1. season (Nairobi seasonal patterns):
season
DRY_SEASON     15193
LONG_RAINS      7982
SHORT_RAINS     7889
Name: count, dtype: int64

2. rain_risk_score (0-1, higher = more likely rain):
   Range: 0.15 to 0.85
   Mean: 0.46

   HIGH severity rate by season:
   - DRY_SEASON     : 12.58%
   - LONG_RAINS     : 12.55%
   - SHORT_RAINS    : 12.31%

CREATING VISIBILITY FEATURES

3. daylight_status (visibility conditions):
daylight_status
DAYLIGHT    20756
DARKNESS    10308
Name: count, dtype: int64

   HIGH severity rate by visibility:
   - DAYLIGHT  : 12.21%
   - DARKNESS  : 13.10%

CREATING COMPOUND WEATHER RISK FEATURES

4. rain_darkness_risk (rainy season + night):
   Compound risk cases: 3,549
   HIGH rate with compound risk: 13.38%
   HIGH rate baseline: 12.39%
   Risk multiplier: 1.08x

WEATHER FEATURES SUMMARY

Created 5 weather features:
  1. season
  2. rain_risk_score
  3. daylight_status
  4. high_rain_risk
  5. r

In [6]:
"""
SECTION 5B: Road Infrastructure Feature Engineering

Road infrastructure significantly impacts accident severity:
- Major highways → higher speeds → more severe crashes
- Intersections → conflict points → higher crash risk
- Residential roads → lower speeds → less severe crashes

Data Source Strategy:
Derive road characteristics from:
1. Crash density patterns (high-volume roads vs low-volume)
2. Location clustering (intersections have more crashes)
3. Geographic patterns (highway corridors vs neighborhoods)

In operational deployment, these would come from:
- OpenStreetMap API (road type, intersection presence)
- Google Maps API (speed limits, traffic patterns)
- Pre-loaded GIS database

For this analysis, I used crash patterns as proxies for road types.
"""

print("="*70)
print("ROAD INFRASTRUCTURE FEATURE ENGINEERING")
print("="*70)

# ============================================================================
# CRASH VOLUME AS ROAD TYPE PROXY
# ============================================================================
print("\n" + "="*70)
print("ROAD TYPE CLASSIFICATION (Based on Crash Patterns)")
print("="*70)

def classify_road_type(crashes_at_location):
    """
    Classify road type based on crash volume at location
    
    Logic:
    - Very high crash volume (>200) → Major highway/arterial
    - High crash volume (100-200) → Main road
    - Medium crash volume (20-100) → Secondary road
    - Low crash volume (<20) → Residential/minor road
    """
    if crashes_at_location >= 200:
        return 'MAJOR_HIGHWAY'
    elif crashes_at_location >= 100:
        return 'MAIN_ROAD'
    elif crashes_at_location >= 20:
        return 'SECONDARY_ROAD'
    else:
        return 'RESIDENTIAL'

df['road_type_proxy'] = df['crashes_at_location'].apply(classify_road_type)

print("\n1. road_type_proxy (derived from crash volume):")
print(df['road_type_proxy'].value_counts().sort_index())

# Analyze severity by road type
road_type_severity = df.groupby('road_type_proxy')['severity_binary'].apply(
    lambda x: (x == 'HIGH').sum() / len(x) * 100
)
print("\n   HIGH severity rate by road type:")
for road_type in ['RESIDENTIAL', 'SECONDARY_ROAD', 'MAIN_ROAD', 'MAJOR_HIGHWAY']:
    if road_type in road_type_severity.index:
        print(f"   - {road_type:17s}: {road_type_severity[road_type]:6.2f}%")



# ============================================================================
# INTERSECTION INDICATOR (Crash Clustering)
# ============================================================================
print("\n" + "="*70)
print("INTERSECTION DETECTION (Based on Spatial Clustering)")
print("="*70)

# Locations with very high crash density likely indicate intersections
# We already have crashes_at_location from Notebook 01

# Intersection proxy: locations with >50 crashes (statistical hotspot)
df['likely_intersection'] = (df['crashes_at_location'] > 50).astype(int)

intersection_high_rate = (df[df['likely_intersection'] == 1]['severity_binary'] == 'HIGH').mean() * 100
non_intersection_high_rate = (df[df['likely_intersection'] == 0]['severity_binary'] == 'HIGH').mean() * 100

print("\n2. likely_intersection (crash density >50):")
print(f"   Intersection locations: {df['likely_intersection'].sum():,} crashes ({df['likely_intersection'].mean()*100:.1f}%)")
print(f"   Intersection HIGH rate: {intersection_high_rate:.2f}%")
print(f"   Non-intersection HIGH rate: {non_intersection_high_rate:.2f}%")


# ============================================================================
# SPEED-RELATED RISK (High-Volume Roads)
# ============================================================================
print("\n" + "="*70)
print("SPEED-RELATED RISK INDICATORS")
print("="*70)

# High-speed road indicator (major highways + main roads)
df['high_speed_road'] = df['road_type_proxy'].isin(['MAJOR_HIGHWAY', 'MAIN_ROAD']).astype(int)

high_speed_high_rate = (df[df['high_speed_road'] == 1]['severity_binary'] == 'HIGH').mean() * 100
low_speed_high_rate = (df[df['high_speed_road'] == 0]['severity_binary'] == 'HIGH').mean() * 100

print("\n3. high_speed_road (major highways + main roads):")
print(f"   High-speed road crashes: {df['high_speed_road'].sum():,} ({df['high_speed_road'].mean()*100:.1f}%)")
print(f"   High-speed HIGH rate: {high_speed_high_rate:.2f}%")
print(f"   Low-speed HIGH rate: {low_speed_high_rate:.2f}%")



# ============================================================================
# GEOGRAPHIC RISK ZONES (Central vs Peripheral)
# ============================================================================
print("\n" + "="*70)
print("GEOGRAPHIC RISK ZONES")
print("="*70)

# Calculate distance from Nairobi CBD (approximately -1.2864, 36.8172)
nairobi_cbd_lat = -1.2864
nairobi_cbd_lon = 36.8172

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate distance between two points in kilometers
    """
    from math import radians, sin, cos, sqrt, atan2
    
    R = 6371  # Earth radius in kilometers
    
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    
    return distance

df['distance_from_cbd_km'] = df.apply(
    lambda row: haversine_distance(row['latitude'], row['longitude'], 
                                   nairobi_cbd_lat, nairobi_cbd_lon),
    axis=1
)

# Classify into zones
def classify_zone(distance_km):
    """
    Classify location into geographic zones
    """
    if distance_km < 5:
        return 'CBD_CORE'
    elif distance_km < 15:
        return 'INNER_SUBURBS'
    elif distance_km < 30:
        return 'OUTER_SUBURBS'
    else:
        return 'PERIPHERAL'

df['geographic_zone'] = df['distance_from_cbd_km'].apply(classify_zone)

print("\n4. geographic_zone (distance from Nairobi CBD):")
print(df['geographic_zone'].value_counts().sort_index())

# Analyze severity by zone
zone_severity = df.groupby('geographic_zone')['severity_binary'].apply(
    lambda x: (x == 'HIGH').sum() / len(x) * 100
)
print("\n   HIGH severity rate by zone:")
for zone in ['CBD_CORE', 'INNER_SUBURBS', 'OUTER_SUBURBS', 'PERIPHERAL']:
    if zone in zone_severity.index:
        print(f"   - {zone:17s}: {zone_severity[zone]:6.2f}%")

print("\n5. distance_from_cbd_km (continuous):")
print(f"   Range: {df['distance_from_cbd_km'].min():.2f} to {df['distance_from_cbd_km'].max():.2f} km")
print(f"   Mean: {df['distance_from_cbd_km'].mean():.2f} km")
print(f"   Median: {df['distance_from_cbd_km'].median():.2f} km")



# ============================================================================
# COMPOUND INFRASTRUCTURE RISK
# ============================================================================
print("\n" + "="*70)
print("COMPOUND INFRASTRUCTURE RISK")
print("="*70)

# High-risk infrastructure: intersection + high-speed road
df['high_risk_infrastructure'] = ((df['likely_intersection'] == 1) & 
                                   (df['high_speed_road'] == 1)).astype(int)

infra_risk_high_rate = (df[df['high_risk_infrastructure'] == 1]['severity_binary'] == 'HIGH').mean() * 100
baseline_infra_rate = (df[df['high_risk_infrastructure'] == 0]['severity_binary'] == 'HIGH').mean() * 100

print("\n6. high_risk_infrastructure (intersection × high-speed road):")
print(f"   High-risk infrastructure: {df['high_risk_infrastructure'].sum():,} crashes")
print(f"   HIGH rate with risk: {infra_risk_high_rate:.2f}%")
print(f"   HIGH rate baseline: {baseline_infra_rate:.2f}%")
if baseline_infra_rate > 0:
    print(f"   Risk multiplier: {infra_risk_high_rate/baseline_infra_rate:.2f}x")



# ============================================================================
# SUMMARY OF ROAD INFRASTRUCTURE FEATURES
# ============================================================================
print("\n" + "="*70)
print("ROAD INFRASTRUCTURE FEATURES SUMMARY")
print("="*70)

road_features = [
    'road_type_proxy',
    'likely_intersection',
    'high_speed_road',
    'geographic_zone',
    'distance_from_cbd_km',
    'high_risk_infrastructure'
]

print(f"\n Created {len(road_features)} road infrastructure features:")
for i, feature in enumerate(road_features, 1):
    print(f"  {i}. {feature}")

print(f"\n Road infrastructure features engineered")
print(f" Total features now: {len(df.columns)}")
print(f" Feature engineering complete!")
print(f"\n Ready for train/validation/test split")

ROAD INFRASTRUCTURE FEATURE ENGINEERING

ROAD TYPE CLASSIFICATION (Based on Crash Patterns)

1. road_type_proxy (derived from crash volume):
road_type_proxy
MAIN_ROAD          4752
MAJOR_HIGHWAY     15376
RESIDENTIAL        2601
SECONDARY_ROAD     8335
Name: count, dtype: int64

   HIGH severity rate by road type:
   - RESIDENTIAL      :  13.30%
   - SECONDARY_ROAD   :  12.11%
   - MAIN_ROAD        :  12.44%
   - MAJOR_HIGHWAY    :  12.60%

INTERSECTION DETECTION (Based on Spatial Clustering)

2. likely_intersection (crash density >50):
   Intersection locations: 24,979 crashes (80.4%)
   Intersection HIGH rate: 12.33%
   Non-intersection HIGH rate: 13.23%

SPEED-RELATED RISK INDICATORS

3. high_speed_road (major highways + main roads):
   High-speed road crashes: 20,128 (64.8%)
   High-speed HIGH rate: 12.56%
   Low-speed HIGH rate: 12.39%

GEOGRAPHIC RISK ZONES

4. geographic_zone (distance from Nairobi CBD):
geographic_zone
CBD_CORE          9905
INNER_SUBURBS    14771
OUTER_SUBURBS

In [7]:
"""
SECTION 6: Prepare Final Feature Set for Modeling

1. Select only predictive features (remove identifiers)
2. Encode categorical variables
3. Verify no data leakage
4. Prepare for train/validation/test split

Final feature set will include:
- Numeric features (continuous and binary indicators)
- Encoded categorical features (one-hot encoding)
- Target variable (severity_binary)
"""

print("="*70)
print("PREPARING FINAL FEATURE SET FOR MODELING")
print("="*70)


# ============================================================================
# IDENTIFY FEATURE CATEGORIES
# ============================================================================
print("\n" + "="*70)
print("FEATURE CATEGORIZATION")
print("="*70)

# Features to EXCLUDE (identifiers, not predictive)
exclude_features = [
    'crash_id',           # Unique identifier
    'crash_datetime',     # Redundant (hour/day/month extracted)
    'crash_date',         # Redundant
    'day_name',           # Redundant (day_of_week is numeric version)
    'lat_bin',            # Redundant (crashes_at_location derived from this)
    'lon_bin',            # Redundant
    'grid_cell',          # Redundant (string version of lat_bin/lon_bin)
    'severity_4class'     # Not using 4-class, only binary
]

# Numeric features (already in usable format)
numeric_features = [
    # Location
    'latitude',
    'longitude',
    # Temporal original
    'hour',
    'day_of_week',
    'month',
    'year',
    'is_weekend',
    # Temporal engineered
    'hour_severity_rate',
    'day_severity_rate',
    'month_severity_rate',
    'is_night',
    'is_rush_hour',
    # Spatial original
    'crashes_at_location',
    'high_rate_at_location',
    # Spatial engineered
    'high_risk_location',
    'dangerous_time',
    'high_risk_location_dangerous_time',
    # Weather
    'rain_risk_score',
    'high_rain_risk',
    'rain_darkness_risk',
    # Road infrastructure
    'likely_intersection',
    'high_speed_road',
    'distance_from_cbd_km',
    'high_risk_infrastructure'
]

# Categorical features (need encoding)
categorical_features = [
    'location_risk_category',  # 4 categories
    'season',                  # 3 categories
    'daylight_status',         # 2 categories
    'road_type_proxy',         # 4 categories
    'geographic_zone'          # 4 categories
]

# Target variable
target = 'severity_binary'

print(f"\n Features to EXCLUDE: {len(exclude_features)}")
for feature in exclude_features:
    print(f"  - {feature}")

print(f"\nNUMERIC features: {len(numeric_features)}")
for feature in numeric_features:
    print(f"  {feature}")

print(f"\nCATEGORICAL features: {len(categorical_features)}")
for feature in categorical_features:
    print(f" {feature} ({df[feature].nunique()} categories)")

print(f"\n TARGET variable: {target}")



# ============================================================================
# ENCODE CATEGORICAL FEATURES
# ============================================================================
print("\n" + "="*70)
print("ENCODING CATEGORICAL FEATURES")
print("="*70)

# One-hot encode categorical features
df_encoded = pd.get_dummies(df, columns=categorical_features, drop_first=False)

print(f"\nBefore encoding: {len(df.columns)} columns")
print(f"After encoding:  {len(df_encoded.columns)} columns")

# Show encoding results
print("\nEncoding details:")
for feature in categorical_features:
    original_categories = df[feature].nunique()
    encoded_cols = [col for col in df_encoded.columns if col.startswith(feature + '_')]
    print(f"  {feature}: {original_categories} categories → {len(encoded_cols)} binary columns")



# ============================================================================
# CREATE FINAL FEATURE SET
# ============================================================================
print("\n" + "="*70)
print("CREATING FINAL FEATURE MATRIX")
print("="*70)

# Get all encoded categorical column names
encoded_categorical_cols = []
for feature in categorical_features:
    encoded_categorical_cols.extend([col for col in df_encoded.columns if col.startswith(feature + '_')])

# Combine numeric and encoded categorical features
all_features = numeric_features + encoded_categorical_cols

# Create final feature matrix X and target y
X = df_encoded[all_features].copy()
y = df_encoded[target].copy()

# Convert target to binary numeric (0=LOW, 1=HIGH)
y_numeric = (y == 'HIGH').astype(int)

print(f"\nFinal feature matrix shape: {X.shape}")
print(f"  - Samples: {X.shape[0]:,}")
print(f"  - Features: {X.shape[1]}")

print(f"\nTarget distribution:")
print(f"  - LOW (0):  {(y_numeric == 0).sum():,} ({(y_numeric == 0).mean()*100:.2f}%)")
print(f"  - HIGH (1): {(y_numeric == 1).sum():,} ({(y_numeric == 1).mean()*100:.2f}%)")
print(f"  - Imbalance ratio: {(y_numeric == 0).sum() / (y_numeric == 1).sum():.2f}:1")



# ============================================================================
# VERIFY NO MISSING VALUES
# ============================================================================
print("\n" + "="*70)
print("DATA QUALITY FINAL CHECK")
print("="*70)

missing_in_X = X.isnull().sum().sum()
missing_in_y = y_numeric.isnull().sum()

print(f"\nMissing values in features (X): {missing_in_X}")
print(f"Missing values in target (y): {missing_in_y}")

if missing_in_X == 0 and missing_in_y == 0:
    print("\n No missing values - dataset is clean!")
else:
    print("\n WARNING: Missing values detected - need to handle before modeling")



# ============================================================================
# SAVE FEATURE NAMES FOR LATER USE
# ============================================================================
print("\n" + "="*70)
print("FEATURE INVENTORY")
print("="*70)

print(f"\n Total predictive features: {len(all_features)}")
print(f"  - Numeric features: {len(numeric_features)}")
print(f"  - Encoded categorical: {len(encoded_categorical_cols)}")

print(f"\n Feature preparation complete")
print(f" Ready for train/validation/test split")

# Store for next cell
feature_names = all_features

PREPARING FINAL FEATURE SET FOR MODELING

FEATURE CATEGORIZATION

 Features to EXCLUDE: 8
  - crash_id
  - crash_datetime
  - crash_date
  - day_name
  - lat_bin
  - lon_bin
  - grid_cell
  - severity_4class

NUMERIC features: 24
  latitude
  longitude
  hour
  day_of_week
  month
  year
  is_weekend
  hour_severity_rate
  day_severity_rate
  month_severity_rate
  is_night
  is_rush_hour
  crashes_at_location
  high_rate_at_location
  high_risk_location
  dangerous_time
  high_risk_location_dangerous_time
  rain_risk_score
  high_rain_risk
  rain_darkness_risk
  likely_intersection
  high_speed_road
  distance_from_cbd_km
  high_risk_infrastructure

CATEGORICAL features: 5
 location_risk_category (4 categories)
 season (3 categories)
 daylight_status (2 categories)
 road_type_proxy (4 categories)
 geographic_zone (4 categories)

 TARGET variable: severity_binary

ENCODING CATEGORICAL FEATURES

Before encoding: 38 columns
After encoding:  50 columns

Encoding details:
  location_risk_ca

In [8]:
"""
SECTION 7: Train/Validation/Test Split + SMOTE

Proper ML workflow requires:
1. Split into Train (70%), Validation (15%), Test (15%)
2. Apply SMOTE ONLY to training set (prevent data leakage)
3. Keep validation and test sets with natural class distribution

Why this matters:
- Training set: Used to fit model (SMOTE applied here)
- Validation set: Used to tune hyperparameters (natural distribution)
- Test set: Final performance evaluation (natural distribution)

SMOTE must NEVER touch validation or test sets, or we get artificially
inflated performance estimates that won't work in real deployment.
"""

print("="*70)
print("TRAIN/VALIDATION/TEST SPLIT + SMOTE")
print("="*70)

# ============================================================================
# STEP 1: INITIAL SPLIT (Train vs Test)
# ============================================================================
print("\n" + "="*70)
print("STEP 1: INITIAL SPLIT (Train 85% + Test 15%)")
print("="*70)

# First split: separate test set (15%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y_numeric,
    test_size=0.15,
    stratify=y_numeric,  # Maintain class proportions
    random_state=42
)

print(f"\nAfter first split:")
print(f"  - Temp set (train+val): {len(X_temp):,} samples ({len(X_temp)/len(X)*100:.1f}%)")
print(f"  - Test set:             {len(X_test):,} samples ({len(X_test)/len(X)*100:.1f}%)")

print(f"\nTest set class distribution:")
print(f"  - LOW (0):  {(y_test == 0).sum():,} ({(y_test == 0).mean()*100:.2f}%)")
print(f"  - HIGH (1): {(y_test == 1).sum():,} ({(y_test == 1).mean()*100:.2f}%)")



# ============================================================================
# STEP 2: SPLIT TEMP INTO TRAIN AND VALIDATION
# ============================================================================
print("\n" + "="*70)
print("STEP 2: SPLIT TEMP INTO TRAIN (70%) + VALIDATION (15%)")
print("="*70)

# Second split: train (70% of total) vs validation (15% of total)
# From temp (85%), we want: train=70/85=82.35%, val=15/85=17.65%
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=0.176,  # 15% of total = 17.6% of temp
    stratify=y_temp,
    random_state=42
)

print(f"\nAfter second split:")
print(f"  - Train set:      {len(X_train):,} samples ({len(X_train)/len(X)*100:.1f}%)")
print(f"  - Validation set: {len(X_val):,} samples ({len(X_val)/len(X)*100:.1f}%)")
print(f"  - Test set:       {len(X_test):,} samples ({len(X_test)/len(X)*100:.1f}%)")

print(f"\nTrain set class distribution (BEFORE SMOTE):")
print(f"  - LOW (0):  {(y_train == 0).sum():,} ({(y_train == 0).mean()*100:.2f}%)")
print(f"  - HIGH (1): {(y_train == 1).sum():,} ({(y_train == 1).mean()*100:.2f}%)")
print(f"  - Imbalance ratio: {(y_train == 0).sum() / (y_train == 1).sum():.2f}:1")

print(f"\nValidation set class distribution (natural):")
print(f"  - LOW (0):  {(y_val == 0).sum():,} ({(y_val == 0).mean()*100:.2f}%)")
print(f"  - HIGH (1): {(y_val == 1).sum():,} ({(y_val == 1).mean()*100:.2f}%)")



# ============================================================================
# STEP 3: APPLY SMOTE TO TRAINING SET ONLY
# ============================================================================
print("\n" + "="*70)
print("STEP 3: APPLY SMOTE TO TRAINING SET ONLY")
print("="*70)

print("\n  CRITICAL: SMOTE is applied ONLY to training data!")
print("   Validation and test sets keep natural class distribution.")
print("   This ensures realistic performance estimates.\n")

# Initialize SMOTE
smote = SMOTE(random_state=42, k_neighbors=5)

# Apply SMOTE to training set
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print(f"Training set AFTER SMOTE:")
print(f"  - Total samples: {len(X_train_balanced):,} (increased from {len(X_train):,})")
print(f"  - LOW (0):  {(y_train_balanced == 0).sum():,} ({(y_train_balanced == 0).mean()*100:.2f}%)")
print(f"  - HIGH (1): {(y_train_balanced == 1).sum():,} ({(y_train_balanced == 1).mean()*100:.2f}%)")
print(f"  - Imbalance ratio: {(y_train_balanced == 0).sum() / (y_train_balanced == 1).sum():.2f}:1")

synthetic_samples_added = len(X_train_balanced) - len(X_train)
print(f"\n  SMOTE created {synthetic_samples_added:,} synthetic HIGH severity samples")
print(f"  Training set is now perfectly balanced (1:1 ratio)")



# ============================================================================
# FINAL DATASET SUMMARY
# ============================================================================
print("\n" + "="*70)
print("FINAL DATASET SUMMARY")
print("="*70)

print("\n1. TRAINING SET (SMOTE-balanced):")
print(f"   Shape: {X_train_balanced.shape}")
print(f"   LOW:  {(y_train_balanced == 0).sum():,} samples")
print(f"   HIGH: {(y_train_balanced == 1).sum():,} samples")
print(f"   Purpose: Model training with balanced classes")

print("\n2. VALIDATION SET (natural distribution):")
print(f"   Shape: {X_val.shape}")
print(f"   LOW:  {(y_val == 0).sum():,} samples ({(y_val == 0).mean()*100:.2f}%)")
print(f"   HIGH: {(y_val == 1).sum():,} samples ({(y_val == 1).mean()*100:.2f}%)")
print(f"   Purpose: Hyperparameter tuning and model selection")

print("\n3. TEST SET (natural distribution):")
print(f"   Shape: {X_test.shape}")
print(f"   LOW:  {(y_test == 0).sum():,} samples ({(y_test == 0).mean()*100:.2f}%)")
print(f"   HIGH: {(y_test == 1).sum():,} samples ({(y_test == 1).mean()*100:.2f}%)")
print(f"   Purpose: Final unbiased performance evaluation")

print("\n" + "="*70)
print("KEY PRINCIPLES FOLLOWED")
print("="*70)
print("\n Stratified splitting maintains class proportions")
print(" SMOTE applied ONLY to training set (prevents data leakage)")
print(" Validation and test sets have natural class distribution")
print(" Test set completely held out (never seen during training)")
print(" Model will be evaluated on realistic class imbalance")

print("\n Data splitting and balancing complete!")
print(" Ready to save datasets for modeling notebook")

TRAIN/VALIDATION/TEST SPLIT + SMOTE

STEP 1: INITIAL SPLIT (Train 85% + Test 15%)

After first split:
  - Temp set (train+val): 26,404 samples (85.0%)
  - Test set:             4,660 samples (15.0%)

Test set class distribution:
  - LOW (0):  4,077 (87.49%)
  - HIGH (1): 583 (12.51%)

STEP 2: SPLIT TEMP INTO TRAIN (70%) + VALIDATION (15%)

After second split:
  - Train set:      21,756 samples (70.0%)
  - Validation set: 4,648 samples (15.0%)
  - Test set:       4,660 samples (15.0%)

Train set class distribution (BEFORE SMOTE):
  - LOW (0):  19,036 (87.50%)
  - HIGH (1): 2,720 (12.50%)
  - Imbalance ratio: 7.00:1

Validation set class distribution (natural):
  - LOW (0):  4,067 (87.50%)
  - HIGH (1): 581 (12.50%)

STEP 3: APPLY SMOTE TO TRAINING SET ONLY

  CRITICAL: SMOTE is applied ONLY to training data!
   Validation and test sets keep natural class distribution.
   This ensures realistic performance estimates.

Training set AFTER SMOTE:
  - Total samples: 38,072 (increased from 21

In [11]:
"""
SECTION 8: Save Processed Datasets

Save prepared datasets for modeling notebook:
1. Training set (SMOTE-balanced)
2. Validation set (natural distribution)
3. Test set (natural distribution)
4. Feature metadata
"""

print("="*70)
print("SAVING PROCESSED DATASETS")
print("="*70)

import joblib
import os

# Create features directory if it doesn't exist
os.makedirs('../data/features', exist_ok=True)

# ============================================================================
# SAVE DATASETS AS CSV FILES
# ============================================================================
print("\n" + "="*70)
print("SAVING DATASETS AS CSV")
print("="*70)

# Create DataFrames with feature names
train_df = pd.DataFrame(X_train_balanced, columns=feature_names)
train_df['severity_binary'] = y_train_balanced

val_df = pd.DataFrame(X_val, columns=feature_names)
val_df['severity_binary'] = y_val

test_df = pd.DataFrame(X_test, columns=feature_names)
test_df['severity_binary'] = y_test

# Save to CSV
train_df.to_csv('../data/features/train_balanced.csv', index=False)
val_df.to_csv('../data/features/validation.csv', index=False)
test_df.to_csv('../data/features/test.csv', index=False)

print(f"\n Training set:   {train_df.shape[0]:,} rows × {train_df.shape[1]} columns")
print(f" Validation set: {val_df.shape[0]:,} rows × {val_df.shape[1]} columns")
print(f" Test set:       {test_df.shape[0]:,} rows × {test_df.shape[1]} columns")



# ============================================================================
# SAVE NUMPY ARRAYS
# ============================================================================
print("\n" + "="*70)
print("SAVING NUMPY ARRAYS")
print("="*70)

np.save('../data/features/X_train_balanced.npy', X_train_balanced)
np.save('../data/features/y_train_balanced.npy', y_train_balanced)
np.save('../data/features/X_val.npy', X_val)
np.save('../data/features/y_val.npy', y_val)
np.save('../data/features/X_test.npy', X_test)
np.save('../data/features/y_test.npy', y_test)

print("\n✓ Numpy arrays saved")
print("  - X_train_balanced.npy, y_train_balanced.npy")
print("  - X_val.npy, y_val.npy")
print("  - X_test.npy, y_test.npy")



# ============================================================================
# SAVE FEATURE METADATA
# ============================================================================
print("\n" + "="*70)
print("SAVING FEATURE METADATA")
print("="*70)

feature_metadata = {
    'feature_names': feature_names,
    'n_features': len(feature_names),
    'numeric_features': numeric_features,
    'categorical_features': categorical_features,
    'n_train': len(X_train_balanced),
    'n_val': len(X_val),
    'n_test': len(X_test)
}

joblib.dump(feature_metadata, '../data/features/feature_metadata.pkl')

print(f"\n✓ Feature metadata saved")
print(f"  - {len(feature_names)} feature names")
print(f"  - Dataset sizes recorded")



# ============================================================================
# FINAL SUMMARY
# ============================================================================
print("\n" + "="*70)
print("NOTEBOOK 02 COMPLETE")
print("="*70)

print("\n Data cleaning & feature engineering complete!")
print(f" 41 dispatch-time features created")
print(f" {len(X_train_balanced):,} training samples (SMOTE-balanced)")
print(f" {len(X_val):,} validation samples")
print(f" {len(X_test):,} test samples")
print("\n Ready for model training in Notebook 03!")

SAVING PROCESSED DATASETS

SAVING DATASETS AS CSV

 Training set:   38,072 rows × 42 columns
 Validation set: 4,648 rows × 42 columns
 Test set:       4,660 rows × 42 columns

SAVING NUMPY ARRAYS

✓ Numpy arrays saved
  - X_train_balanced.npy, y_train_balanced.npy
  - X_val.npy, y_val.npy
  - X_test.npy, y_test.npy

SAVING FEATURE METADATA

✓ Feature metadata saved
  - 41 feature names
  - Dataset sizes recorded

NOTEBOOK 02 COMPLETE

 Data cleaning & feature engineering complete!
 41 dispatch-time features created
 38,072 training samples (SMOTE-balanced)
 4,648 validation samples
 4,660 test samples

 Ready for model training in Notebook 03!
